In [1]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maxdumas. Use `wandb login --relogin` to force relogin


In [2]:
import sagemaker
import boto3
boto3_session = boto3.Session(profile_name="cornell")
session = sagemaker.Session(boto_session=boto3_session)

In [6]:
from sagemaker.pytorch import PyTorch

# This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
job_name = "clip-forge-latent-flows"
print("Job name: ", job_name)

# This is the location that SageMaker will automatically store (and load)
# checkpoints from. We can use this to automatically resume training from an
# earlier checkpoint when using Spot instances.
checkpoint_bucket = session.default_bucket()
autoresume_checkpoint_prefix = f"{job_name}/01" # IMPORTANT: Update this prefix if you don't want to automatically resume training from a previous checkpoint
autoresume_checkpoint_s3_uri = f"s3://{checkpoint_bucket}/{autoresume_checkpoint_prefix}"
print(f"Checkpoints resumable from {autoresume_checkpoint_s3_uri}")

estimator = PyTorch(
    base_job_name=job_name,
    source_dir=".",
    entry_point="train_post_clip.py",  #  the entry point that launches the training script with options
    role="arn:aws:iam::870747888580:role/SageMakerTrainingRole",
    sagemaker_session=session,
    framework_version="1.13.1", # PyTorch version to use
    py_version="py39", # Python version to use
    instance_count=1, # Number of instances to launch
    instance_type="ml.g5.8xlarge", # Instance type to launch
    debugger_hook_config=False,
    keep_alive_period_in_seconds=600, # Keep instances alive for 10 minutes after training to quicken iteration
    environment={
        "WANDB_API_KEY": current_api_key,
        # Allow W&B to resume the current run if Spot interrupts and restarts us on a different machine.
        "WANDB_RESUME": "allow",
        "WANDB_RUN_ID": wandb.util.generate_id(),
    },

    input_mode="FastFile",
    checkpoint_s3_uri=autoresume_checkpoint_s3_uri, # S3 location to automatically load and store checkpoints from
    # use_spot_instances=True, # Use Managed Spot Training
    # max_wait=24*60*60+1, # Must be larger than 24 hours, the default max runtime,
    hyperparameters={
        "autoencoder_checkpoint": "maxdumas/clip_forge/model-clip-forge-autoencoder-2023-02-26-06-15-12-280-tpdwmj-algo-1:v151",
        "num_views": "1",
        "text_query": "\"a chair\" \"a limo\" \"a jet plane\"",
        "batch_size": "128",
        "test_batch_size": "128",
        "lr": "0.00003",
    },
)

Job name:  clip-forge-latent-flows
Checkpoints resumable from s3://sagemaker-us-east-1-870747888580/clip-forge-latent-flows/01


In [7]:
estimator.fit({ "train": "s3://cornell-mfd64/text2building/shapenet" })

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: clip-forge-latent-flows-2023-03-02-02-22-05-290


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Spot training job can't retain cluster